In [ ]:
# %matplotlib inlineb
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

2021.07.06

计算水汽通量的积分值，在等压面上计算并积分

有冗长的整理部分，通过dict选取变量，并且整理到相应的am，jja上，再进行计算，绘制平均

数据整理部分适用于所有从MPAS中convert出来用于和ERA5I进行环流场上对比的部分

2021.08.16

修改到更大的范围上, 从200hPa开始垂直积分

# 数据读取
读取的ERA5I数据顺便处理为了和MPAS相同的坐标之后再进行的输出

In [ ]:
ds_wind = {}
ds_qv   = {}

# diag数据包含9-1，需要去掉尾部
dir_in = "/raid52/yycheng/MPAS/VR_postprocess/VR_merge_large/ke_daily_vi/"
ds_wind['vr'] = xr.open_mfdataset(dir_in + "*_VR_ke_daily_vi.nc")
ds_wind['vr'] = ds_wind['vr'].sel(Time = ds_wind['vr'].Time.dt.month.isin([4,5,6,7,8]))
dir_in = "/raid52/yycheng/MPAS/VR_postprocess/VR_merge_large/hum_theta_daily_vi/"
ds_qv['vr'] = xr.open_mfdataset(dir_in + "*_VR_hum_theta_daily_vi.nc")
ds_qv['vr'] = ds_qv['vr'].sel(Time = ds_qv['vr'].Time.dt.month.isin([4,5,6,7,8]))

dir_in = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge_large/ke_daily_vi/"
ds_wind['rcm'] = xr.open_mfdataset(dir_in + "*_RCM_ke_daily_vi.nc")
ds_wind['rcm'] = ds_wind['rcm'].sel(Time = ds_wind['rcm'].Time.dt.month.isin([4,5,6,7,8]))
dir_in = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge_large/hum_theta_daily_vi/"
ds_qv['rcm'] = xr.open_mfdataset(dir_in + "*_RCM_hum_theta_daily_vi.nc")
ds_qv['rcm'] = ds_qv['rcm'].sel(Time = ds_qv['rcm'].Time.dt.month.isin([4,5,6,7,8]))

In [ ]:
# 添加ERA5I观测数据
ds_wind['obs'] = {}

# 切换 ERA5I 坐标到MPAS输出结果，方便转换

rename_dict = {"level":"plevels", "lon":"longitude", "lat":"latitude", "time":"Time"}
# ds_wind['obs']['uReconstructZonal'].assign_coords(Time = Time)
# show converting coords
for rename_i in rename_dict:
    print(rename_i + " -----converting to----- " + rename_dict[rename_i])
# 读取数据后就整理坐标，和RCM的坐标一致，之后再进行计算
dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily_large/u_98-17_daily/"
ds_wind['obs']["uReconstructZonal"] = xr.open_dataset(dir_in + "merge_uwnd_98-17_daily.nc")\
    .rename(rename_dict).assign_coords(ds_qv['rcm']['qv'].coords)['uwnd']

dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily_large/v_98-17_daily/"
ds_wind['obs']["uReconstructMeridional"] = xr.open_dataset(dir_in + "merge_vwnd_98-17_daily.nc")\
    .rename(rename_dict).assign_coords(ds_qv['rcm']['qv'].coords)['vwnd']

dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily_large/shum_98-17_daily/"
ds_qv['obs'] = xr.open_dataset(dir_in + "merge_shum_98-17_daily.nc").rename(rename_dict).assign_coords(ds_qv['rcm']['qv'].coords)
ds_qv['obs'] = ds_qv['obs'].rename({"shum":"qv"})

In [ ]:
# ----- select data range -----
lat_sel     = (ds_qv['vr'].latitude >= 5) & (ds_qv['vr'].latitude <= 60)
lon_sel     = (ds_qv['vr'].longitude >= 70) & (ds_qv['vr'].longitude <= 140)
plevels_sel = (ds_qv['vr'].plevels >= 200) & (ds_qv['vr'].plevels <= 925)

time_year    = (ds_qv['vr'].Time.dt.year >= 1998) # 时次相对较长，一开始使用1998年一年进行尝试
time_sel_am     = ds_qv['vr'].Time.dt.month.isin([4,5])
time_sel_jja    = ds_qv['vr'].Time.dt.month.isin([6,7,8])

sel_dict = {}
sel_dict['am']    = {'longitude':lon_sel, "latitude":lat_sel, "plevels":plevels_sel, "Time":(time_sel_am & time_year)}
sel_dict['jja']   = {'longitude':lon_sel, "latitude":lat_sel, "plevels":plevels_sel, "Time":(time_sel_jja & time_year)}

In [ ]:
# ----- 选取变量 -----
mod_list    = ['obs', 'vr', 'rcm']
season_list = ['am', 'jja']
# 存放整理后变量的字典
u_sel = {}
v_sel = {}
qv_sel = {}

for imod in mod_list:
    # print(imod)
    u_sel[imod]  = {}
    v_sel[imod]  = {}
    qv_sel[imod] = {}
    for iseason in season_list:
        # print(iseason)
        u_sel[imod][iseason]  = ds_wind[imod]['uReconstructZonal'].isel(sel_dict[iseason])
        v_sel[imod][iseason]  = ds_wind[imod]['uReconstructMeridional'].isel(sel_dict[iseason])
        qv_sel[imod][iseason] = ds_qv[imod]['qv'].isel(sel_dict[iseason])

# 计算垂直积分

保存到变量 uflux_vi 和 vflux_vi 之中

In [ ]:
# count flux
uflux = {}
vflux = {}
# count flux vertical integration
uflux_vi = {}
vflux_vi = {}
sel_plevels_val = u_sel['vr']['am'].plevels

for imod in mod_list:
    print(imod)
    uflux[imod] = {}
    vflux[imod] = {}
    uflux_vi[imod] = {}
    vflux_vi[imod] = {}
    for iseason in season_list:
        print(iseason)
        uflux[imod][iseason] = u_sel[imod][iseason] * qv_sel[imod][iseason]
        vflux[imod][iseason] = v_sel[imod][iseason] * qv_sel[imod][iseason]
        plevels_axis = list(uflux[imod][iseason].dims).index('plevels')
        # remove nan values
        uflux[imod][iseason] = np.where(np.isnan(uflux[imod][iseason]), 0, uflux[imod][iseason])
        vflux[imod][iseason] = np.where(np.isnan(vflux[imod][iseason]), 0, vflux[imod][iseason])
        # find plevels axis location
        uflux_vi[imod][iseason] = np.trapz(uflux[imod][iseason], sel_plevels_val, axis = plevels_axis)
        vflux_vi[imod][iseason] = np.trapz(vflux[imod][iseason], sel_plevels_val, axis = plevels_axis)
# 

In [ ]:
# 整理绘图所使用的坐标变量
lat  = u_sel['vr']['jja'].latitude
lon  = u_sel['vr']['jja'].longitude
time = u_sel['vr']['jja'].Time
time_am = u_sel['vr']['am'].Time
time_jja = u_sel['vr']['jja'].Time
time_iseason = {}
time_iseason['am'] = time_am
time_iseason['jja'] = time_jja

# 临时文件

进行文件的输出部分 计算时间太长 需要一个半小时

进行flux垂直积分之后结果的输出，放置到 /raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/dyn/shum_flux 之中

合并了RCM的边界上的空缺，但是文件输出都是手动进行的，注意对变量、时次进行调整

In [ ]:
# 处理RCM 中边界的NAN的问题，合并成同样的fillvalues，之后赋值为 -1e30
for iseason in season_list:
    rcm_nan_index = (uflux_vi['rcm'][iseason][:,:,:] > 1e10)
    uflux_vi['rcm'][iseason][rcm_nan_index] = -1e30
    vflux_vi['rcm'][iseason][rcm_nan_index] = -1e30

In [ ]:
# 临时存放变量
# uflux_vi['obs']['am'].shape
for iseason in ['am','jja']:
    uflux_vi_tonc = {}
    vflux_vi_tonc = {}
    encoding = {}
    for imod in mod_list:
        uflux_vi_tonc[imod] = xr.DataArray(uflux_vi[imod][iseason], coords=[time_iseason[iseason],lat,lon], name=imod)
        # 添加NAN时需要
        encoding[imod] = {"_FillValue":-1e30}

    ds_out = xr.merge(list( uflux_vi_tonc.values() ))
    ds_out.fillna(-1e30)

    dir_out = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/dyn/shum_flux/"
    ds_out.to_netcdf(dir_out + "shum_uflux_"+iseason+".nc", encoding = encoding)
    ds_out.close()